In [105]:
import pandas as pd
import duckdb


In [106]:
# Load CSV files into DuckDB
con = duckdb.connect()

In [107]:
# Load data from CSV files with error handling
con.execute("""
CREATE TABLE IF NOT EXISTS ADMISSIONS AS SELECT * FROM read_csv_auto('data/ADMISSIONS.csv', ignore_errors=true);
CREATE TABLE IF NOT EXISTS D_ICD_DIAGNOSES AS SELECT * FROM read_csv_auto('data/D_ICD_DIAGNOSES.csv', ignore_errors=true);
CREATE TABLE IF NOT EXISTS DIAGNOSES_ICD AS SELECT * FROM read_csv_auto('data/DIAGNOSES_ICD.csv', ignore_errors=true);
CREATE TABLE IF NOT EXISTS PATIENTS AS SELECT * FROM read_csv_auto('data/PATIENTS.csv', ignore_errors=true);
CREATE TABLE IF NOT EXISTS PRESCRIPTIONS AS SELECT * FROM read_csv_auto('data/PRESCRIPTIONS.csv', ignore_errors=true);
CREATE TABLE IF NOT EXISTS ICUSTAYS AS SELECT * FROM read_csv_auto('data/ICUSTAYS.csv', ignore_errors=true);
CREATE TABLE IF NOT EXISTS D_ICD_PROCEDURES AS SELECT * FROM read_csv_auto('data/D_ICD_PROCEDURES.csv', ignore_errors=true);
CREATE TABLE IF NOT EXISTS PROCEDURES_ICD AS SELECT * FROM read_csv_auto('data/PROCEDURES_ICD.csv', ignore_errors=true);
""")


In [108]:
# 1. Retrieve the count of diabetes-related hospital admissions
diabetes_admissions = con.execute("""
SELECT COUNT(*) AS diabetes_admissions
FROM ADMISSIONS
JOIN DIAGNOSES_ICD ON ADMISSIONS.hadm_id = DIAGNOSES_ICD.hadm_id
WHERE DIAGNOSES_ICD.icd9_code LIKE '250%';
""").fetchdf()
print(diabetes_admissions.head())


   diabetes_admissions
0                11742


In [109]:
# 2. List the top 5 most common diabetes-related diagnoses
diabetes_diagnoses = con.execute("""
SELECT D_ICD_DIAGNOSES.long_title, COUNT(*) AS diagnosis_count
FROM DIAGNOSES_ICD
JOIN D_ICD_DIAGNOSES ON DIAGNOSES_ICD.icd9_code = D_ICD_DIAGNOSES.icd9_code
WHERE DIAGNOSES_ICD.icd9_code LIKE '250%'
GROUP BY D_ICD_DIAGNOSES.long_title
ORDER BY diagnosis_count DESC
LIMIT 5;
""").fetchdf()
with pd.option_context('display.max_colwidth', None):
  display(diabetes_diagnoses)

,LONG_TITLE,diagnosis_count
0,"Diabetes mellitus without mention of complication, type II or unspecified type, not stated as uncontrolled",9058
1,"Diabetes with neurological manifestations, type II or unspecified type, not stated as uncontrolled",1138
2,"Diabetes with renal manifestations, type II or unspecified type, not stated as uncontrolled",954
3,"Diabetes mellitus without mention of complication, type II or unspecified type, uncontrolled",454
4,"Diabetes with other specified manifestations, type II or unspecified type, not stated as uncontrolled",447


In [110]:
# 3. Find the average length of stay for diabetes patients
avg_length_of_stay = con.execute("""
SELECT AVG(EXTRACT(DAY FROM (dischtime - admittime))) AS avg_length_of_stay
FROM ADMISSIONS
JOIN DIAGNOSES_ICD ON ADMISSIONS.hadm_id = DIAGNOSES_ICD.hadm_id
WHERE DIAGNOSES_ICD.icd9_code LIKE '250%';
""").fetchdf()
print(avg_length_of_stay.head())


   avg_length_of_stay
0            9.245784


In [111]:
# 4. Get the gender distribution of diabetes patients
gender_distribution = con.execute("""
SELECT PATIENTS.gender, COUNT(*) AS count
FROM PATIENTS
JOIN DIAGNOSES_ICD ON PATIENTS.subject_id = DIAGNOSES_ICD.subject_id
WHERE DIAGNOSES_ICD.icd9_code LIKE '250%'
GROUP BY PATIENTS.gender;
""").fetchdf()
print(gender_distribution.head())

  GENDER  count
0      F   7280
1      M   9174


In [112]:
# 5. Identify the top 3 most prescribed medications for diabetes patients
top_medications = con.execute("""
SELECT PRESCRIPTIONS.drug, COUNT(*) AS prescription_count
FROM PRESCRIPTIONS
JOIN ADMISSIONS ON PRESCRIPTIONS.hadm_id = ADMISSIONS.hadm_id
JOIN DIAGNOSES_ICD ON ADMISSIONS.hadm_id = DIAGNOSES_ICD.hadm_id
WHERE DIAGNOSES_ICD.icd9_code LIKE '250%'
GROUP BY PRESCRIPTIONS.drug
ORDER BY prescription_count DESC
LIMIT 3;
""").fetchdf()
print(top_medications.head())

                   DRUG  prescription_count
0               Insulin               89803
1    Potassium Chloride               39150
2  0.9% Sodium Chloride               36670


In [113]:
# 6. Find the percentage of diabetes patients who expired in the hospital
mortality_rate = con.execute("""
SELECT (SUM(hospital_expire_flag) * 100.0 / COUNT(*)) AS mortality_rate
FROM ADMISSIONS
JOIN DIAGNOSES_ICD ON ADMISSIONS.hadm_id = DIAGNOSES_ICD.hadm_id
WHERE DIAGNOSES_ICD.icd9_code LIKE '250%';
""").fetchdf()
print(mortality_rate.head())

   mortality_rate
0        9.930165


In [114]:
# 7. List the most common insurance types for diabetes patients
insurance_types = con.execute("""
SELECT insurance, COUNT(*) AS count
FROM ADMISSIONS
JOIN DIAGNOSES_ICD ON ADMISSIONS.hadm_id = DIAGNOSES_ICD.hadm_id
WHERE DIAGNOSES_ICD.icd9_code LIKE '250%'
GROUP BY insurance
ORDER BY count DESC;
""").fetchdf()
print(insurance_types.head())

    INSURANCE  count
0    Medicare   7462
1     Private   2906
2    Medicaid   1062
3  Government    251
4    Self Pay     61


In [115]:
# 8. Find the number of ICU admissions for diabetes patients
icu_admissions = con.execute("""
SELECT COUNT(*) AS icu_admissions
FROM ICUSTAYS
JOIN ADMISSIONS ON ICUSTAYS.hadm_id = ADMISSIONS.hadm_id
JOIN DIAGNOSES_ICD ON ADMISSIONS.hadm_id = DIAGNOSES_ICD.hadm_id
WHERE DIAGNOSES_ICD.icd9_code LIKE '250%';
""").fetchdf()
print(icu_admissions.head())

   icu_admissions
0           12378


In [116]:
# 9. Retrieve the most common procedures performed on diabetes patients
common_procedures = con.execute("""
SELECT D_ICD_PROCEDURES.long_title, COUNT(*) AS procedure_count
FROM PROCEDURES_ICD
JOIN D_ICD_PROCEDURES ON PROCEDURES_ICD.icd9_code = D_ICD_PROCEDURES.icd9_code
JOIN DIAGNOSES_ICD ON PROCEDURES_ICD.hadm_id = DIAGNOSES_ICD.hadm_id
WHERE DIAGNOSES_ICD.icd9_code LIKE '250%'
GROUP BY D_ICD_PROCEDURES.long_title
ORDER BY procedure_count DESC
LIMIT 5;
""").fetchdf()
with pd.option_context('display.max_colwidth', None):
  display(common_procedures)

,LONG_TITLE,procedure_count
0,"Venous catheterization, not elsewhere classified",5129
1,Insertion of endotracheal tube,2660
2,Transfusion of packed cells,2357
3,Continuous invasive mechanical ventilation for less than 96 consecutive hours,2335
4,Extracorporeal circulation auxiliary to open heart surgery,2333


In [117]:
# 10. Nested query: Find patients with multiple diabetes-related hospitalizations
multiple_hospitalizations = con.execute("""
SELECT diabetes_patients.subject_id, COUNT(diabetes_patients.hadm_id) AS readmission_count
FROM (
    SELECT ADMISSIONS.subject_id, ADMISSIONS.hadm_id
    FROM ADMISSIONS
    JOIN DIAGNOSES_ICD ON ADMISSIONS.hadm_id = DIAGNOSES_ICD.hadm_id
    WHERE DIAGNOSES_ICD.icd9_code LIKE '250%'
) AS diabetes_patients
GROUP BY diabetes_patients.subject_id
HAVING COUNT(diabetes_patients.hadm_id) > 1
ORDER BY readmission_count DESC;
""").fetchdf()

print(multiple_hospitalizations.head())


   SUBJECT_ID  readmission_count
0       13033                 66
1       11861                 56
2        7809                 47
3       58163                 32
4        8799                 31


In [118]:
# Close connection
con.close()